# EE-411 TP session 11

## Part 1: Principal Component Analysis

In this set of exercises we will play a little bit with PCA, first introducing it in a very simple case, and then showing its potential in an interesting example. PCA is fundamentally a dimensionality reduction algorithm, but it can also be useful as a tool for visualization, for noise filtering, for feature extraction and engineering, and much more.

**What you will learn today:** In this notebook, we will see how to implement PCA using our old friend SciKitLearn.

In [ ]:
#We import th usual packages
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as sla

## Introducing PCA

_This excercise relies heavily on an example taken from the Python Data Science Handbook, by Jake VanderPlas_

Principal component analysis is a fast and flexible unsupervised method for dimensionality reduction in data.

 Its behavior is easier to visualize by looking at a two-dimensional dataset. 
 
 Let's consider the following 200 points in a 2D plane:

In [ ]:
rng = np.random.RandomState(0) #initialize seed
X = np.dot(rng.rand(2, 2), rng.randn(2, 200)).T
plt.scatter(X[:, 0], X[:, 1]);

By eye, it is clear that there is a nearly linear relationship between the x and y variables, but the problem setting here is slightly different from the usual linear regression: rather than attempting to predict the y values from the x values, the unsupervised learning problem attempts to learn about the relationship between the x and y values.

In PCA, this relationship is quantified by finding a list of the principal axes in the data, and using those axes to describe the dataset.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(X)

Giving the dataset to the function `fit`, the model learns some quantities from the data, most importantly tha *components* and the *explained variance*:



In [ ]:
pca.components_

In [ ]:
pca.explained_variance_

To see what these numbers mean, let's visualize them as vectors over the input data, using the "components" to define the direction of the vector, and the "explained variance" to define the squared-length of the vector:

In [ ]:
#to draw a vector going from the point v0 to the point v1
def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->',
                    linewidth=2,
                    shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops)

plt.figure(figsize=(14,7))
plt.scatter(X[:, 0], X[:, 1], alpha=0.5)
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v)


These vectors represent the principal axes of the data, and the length of the vector is an indication of how "important" that axis is in describing the distribution of the data—more precisely, it is a measure of the variance of the data when projected onto that axis. The projection of each data point onto the principal axes are the "principal components" of the data.

## Dimensionality reduction 

Using PCA for dimensionality reduction involves zeroing out one or more of the smallest principal components, resulting in a lower-dimensional projection of the data that preserves the maximal data variance.

Here is an example of using PCA as a dimensionality reduction transform, using the same data as before:

In [ ]:
pca = PCA(n_components=1)
pca.fit(X)
X_pca = pca.transform(X)
print("original shape:   ", X.shape)
print("transformed shape:", X_pca.shape)

The transformed data has been reduced to a single dimension. To understand the effect of this dimensionality reduction, we can perform the inverse transform of this reduced data and plot it along with the original data:

In [ ]:
X_new = pca.inverse_transform(X_pca)
plt.figure(figsize=(14,7))
plt.scatter(X[:, 0], X[:, 1], alpha=0.5)
plt.scatter(X_new[:, 0], X_new[:, 1], alpha=0.8)

This makes clear what a PCA dimensionality reduction means: the information along the least important principal axis or axes is removed, leaving only the component(s) of the data with the highest variance. The fraction of variance that is cut out (proportional to the spread of points about the line formed in this figure) is roughly a measure of how much "information" is discarded in this reduction of dimensionality.

This reduced-dimension dataset is in some senses "good enough" to encode the most important relationships between the points: despite reducing the dimension of the data by 50%, the overall relationship between the data points are mostly preserved.

## PCA as Noise filtering

PCA can also be used as a filtering approach for noisy data. The idea is this: any components with variance much larger than the effect of the noise should be relatively unaffected by the noise. So if you reconstruct the data using just the largest subset of principal components, you should be preferentially keeping the signal and throwing out the noise.

Let's see how this looks with the digits data. First we will plot several of the input noise-free data:

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
digits.data.shape

def plot_digits(data):
    fig, axes = plt.subplots(4, 10, figsize=(10, 4),
                             subplot_kw={'xticks':[], 'yticks':[]},
                             gridspec_kw=dict(hspace=0.1, wspace=0.1))
    for i, ax in enumerate(axes.flat):
        ax.imshow(data[i].reshape(8, 8),
                  cmap='binary', interpolation='nearest',
                  clim=(0, 16))
plot_digits(digits.data)

Now lets add some random noise to create a noisy dataset, and re-plot it:

In [ ]:
np.random.seed(42)
noisy = np.random.normal(digits.data, 4)
plot_digits(noisy)

It's clear by eye that the images are noisy, and contain spurious pixels. Let's train a PCA on the noisy data, requesting that the projection preserve 50% of the variance:

In [ ]:
pca = PCA(0.50).fit(noisy)
pca.n_components_

Here 50% of the variance amounts to 12 principal components. Now we compute these components, and then use the inverse of the transform to reconstruct the filtered digits:

In [ ]:
components = pca.transform(noisy)
filtered = pca.inverse_transform(components)
plot_digits(filtered)

A vital part of using PCA in practice is the ability to estimate how many components are needed to describe the data. This can be determined by looking at the `cumulative explained variance ratio` as a function of the number of components:

In [ ]:
pca = PCA().fit(digits.data)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

This curve quantifies how much of the total, 64-dimensional variance is contained within the first N components. For example, we see that with the digits the first 10 components contain approximately 75% of the variance, while you need around 50 components to describe close to 100% of the variance.

### Your turn: Eigenfaces

Download the *Labelled Faces in the Wild* (LFW) people dataset from sklearn. This dataset is a collection of pictures of famous people faces. 

By setting `min_faces_per_person=30`, the extracted dataset will only retain pictures of people that have at least 30 different pictures. 

In [ ]:
from sklearn.datasets import fetch_lfw_people

In [ ]:
# Download the dataset

min_faces_per_person = 30

faces = fetch_lfw_people(min_faces_per_person=min_faces_per_person)

Now we plot the first 10 faces (`dataset.target_images`) together with the corresponding names (`dataset.target_names`) using
`imshow`

In [ ]:
# Print the first 10 faces with the corresponding names

for X, y in zip(faces.images[:10], faces.target[:10]):
    
    print(faces.target_names[y])
    plt.imshow(X)
    plt.show()

In [ ]:
print(f"There are {len(faces.data)} images in total, each composed by {len(faces.data[0])} pixels")

#### 1) In order to obtain a balanced dataset, keep only 30 pictures per person. Then, normalize your data in the range [0, 1].

In [ ]:
# your code here

print(f"Now there are {len(X)} images.")

#### 2) Do the PCA on the new dataset, compute the principal components and print their shape

In [ ]:
# your code here

#### 3) Plot, using `imshow`, the first 5 principal components. How do they look like?

In [ ]:
# your code here

#### 4) Plot the cumulative explained variance. How many principal components do we need in order to explain the majority of the variance of our faces?

In [ ]:
# your code here

## Evaluated Question

#### 5) Compute and plot the projections of four pictures on the first {10, 30, 100, 300, 1000} principal components, using the parameter `n_components` and the method `inverse_transform`.

In [ ]:
# your code here

### Now, we are going to compare the performance of a naive classifier (the predicted label corresponds to the label of the nearest-neighbour image) when using the full pictures and the first 100 PCA projections.

#### 6) First, split your data into a training and a test set.

In [ ]:
# your code here

#### 7) Use `sklearn.neighbors.KNeighbordClassifier` with `n_neighbors=1` to fit your training set. Evaluate the performance using the method score on the test set.

In [ ]:
# your code here

#### 8) Now repeat the training but using only the first 100 principal components. Remember to project the test set also before evaluating the score. Compare with the previous score.

In [ ]:
# your code here

#### 9) When calling `PCA`, set the parameter `whiten=True`. This transformation, called ‘whitening’ and it rescales the principal components to have the same variance. Re-classify the faces and compare the accuracy with the previous ones.

In [ ]:
# your code here